In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import os
import datetime
import time
from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED, FIRST_COMPLETED
import sqlite3
from pandas.io import sql
# 让ssl警告不出现
requests.packages.urllib3.disable_warnings()

In [2]:
# 设置全局变量
# 设置主页
homepage = 'https://www.82maopp.com'
# 要爬取的网页，只爬取其中一个模块，要想爬取其他模块的视频请自行更改模块网址
root_url = 'https://www.82maopp.com/vodtypehtml/28.html'
# 视频保存的目录（绝对路径），不存在则会自动创建
root_dir = 'E:/爬取/视频/猫咪/少女绝美/'
# 加载数据库文件（绝对路径），不存在则会自动创建
database = 'E:/爬取/log/log.db'
# 设置线程数，即可以同时下载视频的任务数量
max_tasks = 8
# 设置视频链接下载失败后的重试次数
retry_download = 5
# 设置头部信息
Sec_Fetch_Mode = 'cors'
User_Agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.81 Safari/537.36 SE 2.X MetaSr 1.0'
headers = {
    'Origin': homepage,
    'Sec-Fetch-Mode': Sec_Fetch_Mode,
    'User-Agent': User_Agent,
    'Connection': 'close'
}
# 是否显示日志信息
is_show = True
# 设置网页超时时间
time_out_seconds = 60
# 判断是否有线程正在合并视频
is_tsToMp4 = False
# 是否开启单点下载
is_download_one = False
# 单点下载链接
temp_urls = [
            #                 'https://www.82maopp.com/vodhtml/26494.html',
#             'https://www.82maopp.com/vodhtml/26476.html',
            'https://www.82maopp.com/vodhtml/26873.html'
            #                 'https://www.82maopp.com/vodhtml/26479.html',
            #                 'https://www.82maopp.com/vodhtml/26482.html'
        ]

In [3]:
# 初始化下载目录
if os.path.exists(root_dir) == False:
    os.makedirs(root_dir)
# 初始化数据库
if os.path.exists(database) == False:
    fd = open(database, mode="w", encoding="utf-8")
    fd.close()
# 初始化数据表
con = sqlite3.connect(database)
cur = con.cursor()
# 初始化日志表
result = sql.read_sql(
    "select count(*)  from sqlite_master where type='table' and name = 'log';",
    con)
if result.iloc[0, 0] == 0:
    sql_str = '''
    CREATE TABLE "log" ("event" TEXT,"status" TEXT,"updated_time" TEXT);
    '''
    con.execute(sql_str)
# 初始化日志表已爬取video的视频表
result = sql.read_sql(
    "select count(*)  from sqlite_master where type='table' and name = 'ts_urls';",
    con)
if result.iloc[0, 0] == 0:
    sql_str = '''
    CREATE TABLE "ts_urls" (
        "click_video_url" TEXT,
        "output_dir" TEXT,
        "ts_url" TEXT,
        "updated_time" TEXT,
        "video_titile" TEXT,
        "video_url" TEXT
    );
    '''
    con.execute(sql_str)
# 初始化日志表已爬取ts的视频表
result = sql.read_sql(
    "select count(*)  from sqlite_master where type='table' and name = 'success_video';",
    con)
if result.iloc[0, 0] == 0:
    sql_str = '''
    CREATE TABLE "success_video" (
        "click_video_url" TEXT,
        "output_dir" TEXT,
        "updated_time" TEXT,
        "video_titile" TEXT,
        "video_url" TEXT
    );
    '''
    con.execute(sql_str)
con.close()

In [4]:
# 日志记录
def insert_log(event,status):
    con = sqlite3.connect(database)
    nowTime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    data_dict = {'updated_time': nowTime, 'event': event,'status':status}
    log = pd.DataFrame([data_dict])
    log.to_sql('log',con=con,if_exists='append',index=False)
    if is_show:
        print(status+nowTime + ':'+ event)
    con.close()
# 插入已成功爬取video的视频
def insert_click_video_url(data_dict):
    con = sqlite3.connect(database)
    nowTime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    data_dict['updated_time'] = nowTime
    data = pd.DataFrame([data_dict])
    data = data.loc[:,['click_video_url','video_url','video_titile','output_dir','updated_time']]
    data.to_sql('success_video',con=con,if_exists='append',index=False)
    con.close()
# 插入已成功爬取ts的视频
def insert_ts_url(data_dict):
    con = sqlite3.connect(database)
    nowTime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    data_dict['updated_time'] = nowTime
    ts_urls = pd.DataFrame([data_dict])
    ts_urls.to_sql('ts_urls',con=con,if_exists='append',index=False)
    con.close()

In [5]:
def get_page_urls(session):
    """
    获取所有翻页页面链接
    """
    # 获取页面数量
    root_response = session.get(root_url,headers=headers,timeout=time_out_seconds,verify=False)
    root_response.encoding = 'utf-8'
    bs = BeautifulSoup(root_response.text,'lxml')
    nums_str = bs.find_all(name='a',attrs={'class':'visible-xs'})[0].text
    nums = nums_str.split('/')
    max_page_num = int(nums[1])
    # 获取所有页面链接
    page_url_list = []
    page_url_list.append(root_url)
    for i in range(2,max_page_num+1):
        page_url = root_url.replace('.html','-'+str(i)+'.html')
        page_url_list.append(page_url)
    # 释放连接
    root_response.close()
    del(root_response)
    return page_url_list

In [6]:
def get_click_video_urls(page_url,session):
    """
    获取线路选择播放视频链接
    """
    page_response = session.get(page_url,headers=headers,timeout=time_out_seconds,verify=False)
    page_response.encoding = 'utf-8'
    bs = BeautifulSoup(page_response.text,'lxml')
    click_video_urls = bs.find(name='div',attrs={'id':'tpl-img-content'}).find_all(name='a')
    click_video_urls = [homepage + i.attrs['href'] for i in click_video_urls]
    # 释放连接
    page_response.close()
    del(page_response)
    return click_video_urls

In [7]:
def get_video_url(session,click_video_url):
    """
    获取播放界面链接
    """
    click_video_response = session.get(click_video_url, headers=headers,timeout=time_out_seconds,verify=False)
    click_video_response.encoding = 'utf-8'
    bs = BeautifulSoup(click_video_response.text,'lxml')
    video_url = bs.find_all(name='a',text='线路一')[0].attrs['href']
    video_url = homepage + video_url
    # 释放连接
    click_video_response.close()
    del(click_video_response)
    return video_url

In [8]:
def get_video_name(session,video_url):
    """
    获取视频名称
    """
    video_response = session.get(video_url, headers=headers,timeout=time_out_seconds,verify=False)
    video_response.encoding = 'utf-8'
    bs = BeautifulSoup(video_response.text,'lxml')
    titiles = bs.find('span').find_all(name='a')
    titiles = [i.text for i in titiles]
    titile = '_'.join(titiles)
    # 释放连接
    video_response.close()
    del(video_response)
    return titile

In [9]:
def get_m3u8_url(session,video_url):
    """
    获取m3u8_url
    """
    video_response = session.get(video_url, headers=headers,timeout=time_out_seconds,verify=False)
    m3u8_url = re.findall('var vHLSurl = \"(.+)\"',video_response.text)[0]
    # 释放连接
    video_response.close()
    del(video_response)
    return m3u8_url
    

In [10]:
def get_ts_list(session,m3u8_url,video_url):
    """
    根据m3u8链接获取ts视频下载链接
    m3u8_url:m3u8链接
    返回值：ts视频链接列表
    """
    # 构建headers
    headers = {
        'Origin': homepage,
        'Referer': video_url,
        'Sec-Fetch-Mode': Sec_Fetch_Mode,
        'User-Agent': User_Agent,
        'Connection': 'close'
    }
    # 获取第一层m3u8的主网址
    regular = re.compile(r'http.+.com')
    m3u8_response1_home = re.findall(regular, m3u8_url)[0]
    # 获取第一层m3u8的主网址与第3行的网址连接变成第二层m3u8网址
    m3u8_response1 = session.get(m3u8_url, headers=headers,timeout=time_out_seconds,verify=False)
    m3u8_url2 = m3u8_response1_home + m3u8_response1.text.splitlines()[2]
    # 获取ts链接
    m3u8_response2 = session.get(m3u8_url2, headers=headers,timeout=time_out_seconds,verify=False)
    ts_string_list = m3u8_response2.text.split('\n')
    ts_url_list = []
    for ts_string in ts_string_list:
        # 去除注释行
        if re.match('#', ts_string) != None:
            continue
        # 去除空值
        elif ts_string == '':
            continue
        else:
            ts_url_list.append(m3u8_response1_home + ts_string)
    # 释放连接
    m3u8_response1.close()
    del(m3u8_response1)
    m3u8_response2.close()
    del(m3u8_response2)
    return ts_url_list

In [11]:
def loadTs(session,ls, data_dict):
    """
    下载ts
    ls:ts下载链接列表
    output_dir:输出目录
    """
    headers = {
        'Origin': homepage,
        'Referer': data_dict['video_url'],
        'Sec-Fetch-Mode': Sec_Fetch_Mode,
        'User-Agent': User_Agent,
        'Connection': 'close'
    }
    length = len(ls)
    # 创建文件夹
    if not os.path.exists(data_dict['output_dir']):
        os.mkdir(data_dict['output_dir'])
    # 获取ts链接列表
    con = sqlite3.connect(database)
    ts_urls = sql.read_sql('select * from ts_urls',con)
    con.close()
    for i in range(length):
        # 判断该ts是否已经下载
        if ls[i] in ts_urls['ts_url'].values:
            continue
        else:
            tsname = ls[i][-11:]
            path = os.path.join(data_dict['output_dir'], tsname)
            r = session.get(ls[i], headers=headers,timeout=time_out_seconds)
            with open(path, 'wb') as f:
                f.write(r.content)
                f.close()
                print('\r' + data_dict['video_titile'] +tsname + " -->OK ({}/{}){:.2f}%".format(
                    i, length, i * 100 / length),
                      end='')
            # ts下载完毕就做好记录
            data_dict['ts_url'] = ls[i]
            insert_ts_url(data_dict)
            # 释放连接
            r.close()
            del(r)

In [12]:
def tsToMp4(ts_url_list, data_dict):
    """
    整合所有ts文件，保存为mp4格式
    video_dir：ts目录路径
    """
    # cd到输出文件夹
    os.chdir(data_dict['output_dir'])
    # 设置输出文件名字，不要是ts结尾的
    output_file = 'new.mp4'
    # 如果已经存在输出文件则直接返回
    if os.path.exists(output_file):
        insert_log(data_dict['click_video_url']+data_dict['video_titile']+'已存在输出文件','成功')
        return
    # 设置n个数量文件进行分割数组
    n = 200
    # 获取数组长度
    length = len(ts_url_list)
    # 等份切割数组并保存到data_list数组里面
    data_list = []
    for i in range(0,length,n):
        tem_dict = {}
        # 判断结束位置是否超过数组长度，以数组长度为结束位置
        if i+n >= length:
            filenames = [ts_url_list[j][-11:] for j in range(i,length)]
        else:
            filenames = [ts_url_list[j][-11:] for j in range(i,i+n)]
        filenames_str = '+'.join(filenames)
        # 设置每组的合并后的文件名
        new_filename = '{}.ts'.format(str(i))
        tem_dict = {'new_filename':new_filename,'filenames_str':filenames_str}
        data_list.append(tem_dict)
    # 执行命令合并每组ts文件
    for data in data_list:
        result_code = os.system("copy /b {} {}".format(data['filenames_str'],data['new_filename']))
        # 判断执行是否成功
        if result_code == 0:
            insert_log(data_dict['click_video_url']+data['new_filename']+'合并成功','成功')
        else:
            raise Exception(data_dict['click_video_url']+data['new_filename']+'合并失败')
            return
    # 将每组合并后的ts新文件合并成一个文件
    new_filenames = [i['new_filename'] for i in data_list]
    new_filenames_str = '+'.join(new_filenames)
    result_code = os.system("copy /b {} {}".format(new_filenames_str,output_file))
    # 判断执行是否成功
    if result_code == 0:
        insert_log(data_dict['click_video_url']+output_file+'合并成功','成功')
    else:
        raise Exception(data_dict['click_video_url']+output_file+'合并失败')
        return
    # 删除ts文件
    if os.path.exists(output_file):
        for path in os.listdir(data_dict['output_dir']):
            if path.endswith(".ts"):
                os.remove(path)
    return

In [13]:
def download_job(click_video_url):
    # 单点下载判断
    if is_download_one:
        if click_video_url not in temp_urls:
            return
    # 获取选择线路链接列表
    con = sqlite3.connect(database)
    success_video = sql.read_sql('select * from success_video', con)
    con.close()
    # 判断该链接是否下载完成
    if (click_video_url in success_video['click_video_url'].values):
        insert_log(click_video_url+'已下载完成','成功')
        return
    # 开始下载
    i = 1
    while True:
        if i == i+retry_download:
            insert_log(click_video_url  + '下载失败', '错误')
            return
        try:
            # 创建一个会话
            session = requests.session()
            # 保持连接
            session.keep_alive = True
            
            insert_log('开始解析线路选择播放视频页面：' + click_video_url, '成功')
            # 获取播放界面链接
            video_url = get_video_url(session,click_video_url)
            insert_log('获取播放界面链接：' + video_url, '成功')
            # 获取视频名称
            video_titile = get_video_name(session,video_url)
            insert_log('获取视频名称：' + video_titile, '成功')
            # 获取视频m3u8_url
            m3u8_url = get_m3u8_url(session,video_url)
            insert_log('获取视频m3u8_url：' + m3u8_url, '成功')
            # 获取ts视频流链接
            ts_url_list = get_ts_list(session,m3u8_url, video_url)
            insert_log('获取ts视频流链接：' + video_url, '成功')
            # 构造视频保存文件夹
            output_dir = os.path.join(root_dir, video_titile)
            insert_log('构造视频保存文件夹：' + output_dir, '成功')
            # 开始下载ts视频
            data_dict = {
                'click_video_url': click_video_url,
                'video_url': video_url,
                'video_titile': video_titile,
                'output_dir': output_dir
            }
            loadTs(session,ts_url_list, data_dict)
            insert_log('下载ts视频完毕：' + video_url, '成功')
            # 把ts视频合并成一个MP4格式,需要获取锁
            global is_tsToMp4
            # 判断有没有别的线程正在执行合并视频操作
            while True:
                if is_tsToMp4 == True:
                    insert_log('有线程在合并视频','成功')
                    continue
                else:
                    insert_log('无线程在合并视频','成功')
                    break
            # 将is_tsToMp4变成True，通知所有线程该线程正在执行合并视频操作,加锁
            is_tsToMp4 = True
            tsToMp4(ts_url_list, data_dict)
            # 解锁
            is_tsToMp4 = False
            insert_log('把ts视频合并成一个MP4格式完毕：' + video_url, '成功')
            # 记录该视频已下载保存成功
            insert_click_video_url(data_dict)
            insert_log(data_dict['video_titile'] + data_dict['click_video_url']+'下载保存成功','成功')
            # 退出下载
            break
        except Exception as e:
            # 解锁
            is_tsToMp4 = False
            insert_log('重试'+ str(i)+ ": " +click_video_url  + str(e), '错误')
            # 释放会话
            session.close()
            time.sleep(20)
            i = i+1
            # 下载重试
            continue
            
    return

In [ ]:
if __name__ == '__main__':
    try:
        # 创建一个会话
        session = requests.session()
        # 保持连接
        session.keep_alive = True
        # 获取所有翻页页面链接
        page_url_list = get_page_urls(session)
        insert_log('获取所有翻页页面链接', '成功')
        # 创建4个线程
        threadPool = ThreadPoolExecutor(max_workers=max_tasks,
                                        thread_name_prefix="test_")
        # 一页一页的进行解析并多线程下载视频
        for page_url in page_url_list:
            insert_log('正在解析并下载页面链接：' + page_url, '成功')
            # 解析页面获取该页面所有的视频链接
            click_video_urls = get_click_video_urls(page_url,session)
            # 进行该页面的所有视频链接的多线程解析和下载
            all_task = [threadPool.submit(download_job, click_video_url) for click_video_url in click_video_urls]
            # 等待该页面所有视频下载完毕再进行下一个页面的解析
            wait(all_task, return_when=ALL_COMPLETED)
            insert_log('翻页页面：' + page_url + ' 视频下载成功', '成功')
        # 释放会话
        session.close()
        
        # 等待线程跑完
        threadPool.shutdown(wait=True)

    except Exception as e:
        insert_log(str(e), '错误')

成功2020-07-06 20:55:28:获取所有翻页页面链接
成功2020-07-06 20:55:28:正在解析并下载页面链接：https://www.82maopp.com/vodtypehtml/28.html
成功2020-07-06 20:55:29:https://www.82maopp.com/vodhtml/26885.html已下载完成
成功2020-07-06 20:55:29:https://www.82maopp.com/vodhtml/26857.html已下载完成
成功2020-07-06 20:55:29:https://www.82maopp.com/vodhtml/26860.html已下载完成
成功2020-07-06 20:55:29:https://www.82maopp.com/vodhtml/26497.html已下载完成
成功2020-07-06 20:55:29:https://www.82maopp.com/vodhtml/26500.html已下载完成
成功2020-07-06 20:55:29:https://www.82maopp.com/vodhtml/26488.html已下载完成
成功2020-07-06 20:55:29:https://www.82maopp.com/vodhtml/26876.html已下载完成
成功2020-07-06 20:55:30:https://www.82maopp.com/vodhtml/26494.html已下载完成
成功2020-07-06 20:55:30:https://www.82maopp.com/vodhtml/26476.html已下载完成
成功2020-07-06 20:55:30:https://www.82maopp.com/vodhtml/26479.html已下载完成
成功2020-07-06 20:55:29:https://www.82maopp.com/vodhtml/26879.html已下载完成
成功2020-07-06 20:55:29:https://www.82maopp.com/vodhtml/26491.html已下载完成
成功2020-07-06 20:55:30:https://www.82maopp.com/vod

成功2020-07-06 20:55:48:开始解析线路选择播放视频页面：https://www.82maopp.com/vodhtml/77104.html
成功2020-07-06 20:55:48:开始解析线路选择播放视频页面：https://www.82maopp.com/vodhtml/77102.html
成功2020-07-06 20:55:47:https://www.82maopp.com/vodhtml/77435.html已下载完成
成功2020-07-06 20:55:48:https://www.82maopp.com/vodhtml/77106.html已下载完成
成功2020-07-06 20:55:48:https://www.82maopp.com/vodhtml/77108.html已下载完成
成功2020-07-06 20:55:48:https://www.82maopp.com/vodhtml/77110.html已下载完成
成功2020-07-06 20:55:49:https://www.82maopp.com/vodhtml/77112.html已下载完成
成功2020-07-06 20:55:49:https://www.82maopp.com/vodhtml/76836.html已下载完成
成功2020-07-06 20:55:49:https://www.82maopp.com/vodhtml/76839.html已下载完成
成功2020-07-06 20:55:49:https://www.82maopp.com/vodhtml/76826.html已下载完成
成功2020-07-06 20:55:49:https://www.82maopp.com/vodhtml/76828.html已下载完成
成功2020-07-06 20:55:49:开始解析线路选择播放视频页面：https://www.82maopp.com/vodhtml/76830.html
成功2020-07-06 20:55:49:获取播放界面链接：https://www.82maopp.com/vodplayhtml/77102.html?road=1
成功2020-07-06 20:55:47:https://www.82maopp.com

首页_少女绝美_彼氏に日間禁欲させられケダモノになった巨乳をおれが先に寝取ってめちゃめちゃヤリまくってやった 吉根ゆりあ4yiYvZwg.ts -->OK (347/1803)19.25%成功2020-07-06 21:16:50:获取播放界面链接：https://www.82maopp.com/vodplayhtml/77104.html?road=1
成功2020-07-06 21:16:57:获取视频名称：首页_少女绝美_彼女が家族旅行で一週間留守にしたので彼女の巨乳女友達に中出ししまくりました。 佐知子
首页_少女绝美_ムッチリ総合病院 川原かなえ篠崎かんな桜庭ひかりExvIJR1b.ts -->OK (1301/1596)81.52%8%成功2020-07-06 21:17:00:获取视频m3u8_url：https://bobolj.com/20200326/mnFfeBLe/index.m3u8
成功2020-07-06 21:17:03:获取ts视频流链接：https://www.82maopp.com/vodplayhtml/77104.html?road=1
成功2020-07-06 21:17:03:构造视频保存文件夹：E:/爬取/视频/猫咪/少女绝美/首页_少女绝美_彼女が家族旅行で一週間留守にしたので彼女の巨乳女友達に中出ししまくりました。 佐知子
首页_少女绝美_彼氏に日間禁欲させられケダモノになった巨乳をおれが先に寝取ってめちゃめちゃヤリまくってやった 吉根ゆりあ2BCdyQWI.ts -->OK (364/1803)20.19%错误2020-07-06 21:18:08:重试1: https://www.82maopp.com/vodhtml/77102.htmlHTTPSConnectionPool(host='bobolj.com', port=443): Max retries exceeded with url: /20200326/gAlXdoyA/550kb/hls/42MxW1Ql.ts (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')",),))
错误2020-07-06 21:18:09:重试1: htt